# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for AAPL to the Data Analyst, and ask the Trading Strategy Developer to use statistical modeling and machine learning to identify trends and predict market movements for AAPL.

Action: Delegate work to co-worker
Action Input: 
{
  "coworker": "Data Analyst",
  "task": "Continuously monitor and analyze market data for AAPL",
  "context": "The Data Analyst needs to monitor and analyze market data for AAPL on a regular basis, looking for trends and patterns that could be used to make informed decisions."
}


> Entering new CrewAgentExecutor chain...
I need to start by gathering the most recent market data for AAPL so I can analyze it for

Final Answer: Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance
 News Today's news US Politics World Tech Reviews and deals Audio Computing Gaming Health Home Phones Science TVs Climate change Health Science 2024 election Originals The 360 Life Health COVID-19 Fall allergies Health news Mental health Relax Sexual health Studies The Unwind Parenting Family health So mini ways Style and beauty It Figures Unapologetically Horoscopes Shopping Buying guides Food Travel Autos Gift ideas Buying guides Entertainment Celebrity TV Movies Music How to Watch Interviews Videos Shopping Finance My Portfolio News Latest News Stock Market Originals Premium News Economics Housing Earnings Tech Crypto Biden Economy EV Deep Dive Markets Stocks: Most Actives Stocks: Gainers Stocks: Losers Trending Tickers Futures World Indices US Treasury Bonds Rates Currencies Crypto Top ETFs Top Mutual Funds Options: Highest Open Interest Options: Highest Implied Volatility Sectors Basic Materials Com

Final Answer: 

AAPL Stock Price, News, Quote & History - Yahoo Finance
Today's news US Politics World Tech Reviews and deals Audio Computing Gaming Health Home Phones Science TVs Climate change Health Science 2024 election Originals The 360 Life Health COVID-19 Fall allergies Health news Mental health Relax Sexual health Studies The Unwind Parenting Family health So mini ways Style and beauty It Figures Unapologetically Horoscopes Shopping Buying guides Food Travel Autos Gift ideas Buying guides Entertainment Celebrity TV Movies Music How to Watch Interviews Videos Shopping Finance My Portfolio News Latest News Stock Market Originals Premium News Economics Housing Earnings Tech Crypto Biden Economy EV Deep Dive Markets Stocks: Most Actives Stocks: Gainers Stocks: Losers Trending Tickers Futures World Indices US Treasury Bonds Rates Currencies Crypto Top ETFs Top Mutual Funds Options: Highest Open Interest Options: Highest Implied Volatility Sectors Basic Materials Communication Servic

I need to develop and refine trading strategies for AAPL based on insights from the Data Analyst and user-defined risk tolerance.

Action: Delegate work to co-worker
Action Input: {"coworker": "Trading Strategy Developer", "task": "Develop and refine trading strategies for AAPL based on insights from the Data Analyst and user-defined risk tolerance", "context": "AAPL Stock Price, News, Quote & History - Yahoo Finance"}

> Entering new CrewAgentExecutor chain...
I should start by reading the website content of AAPL Stock Price, News, Quote & History on Yahoo Finance to gather all the necessary information for developing and refining trading strategies.

Action: Read website content
Action Input: {"website_url": "https://finance.yahoo.com/quote/AAPL"} 

Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance
 News Today's news US Politics World Tech Reviews and deals Audio Computing Gaming Health Home Phones Science TVs Climate change Health Science 2024 election Originals T

Final Answer: 
After analyzing the website content of AAPL Stock Price, News, Quote & History on Yahoo Finance, I have gathered valuable information about Apple Inc. (AAPL). This includes details about the company's stock performance, key events, financial statistics, recent news, and comparisons to other companies in the consumer electronics industry. Based on this information, I will now proceed to develop and refine trading strategies for AAPL, considering the insights provided and the user-defined risk tolerance.

> Finished chain.
 

After analyzing the website content of AAPL Stock Price, News, Quote & History on Yahoo Finance, I have gathered valuable information about Apple Inc. (AAPL). This includes details about the company's stock performance, key events, financial statistics, recent news, and comparisons to other companies in the consumer electronics industry. Based on this information, I will now proceed to develop and refine trading strategies for AAPL, considering the in

Final Answer: Page Not Found

> Finished chain.
 

Page Not Found

Thought:
It seems like there was an issue with delegating the task to the Data Analyst. I need to ask a question to another co-worker to ensure the task is completed effectively.

Action: Ask question to co-worker
Action Input: {"coworker": "Trading Strategy Developer", "question": "Can you provide detailed execution plans suggesting how and when to execute trades for AAPL based on the approved trading strategies?", "context": "Need detailed execution plans based on the long-term investment strategy, swing trading strategy, and options trading strategy for AAPL, considering the user's risk tolerance and aligning with current market conditions and optimal pricing."}

> Entering new CrewAgentExecutor chain...
I should start by gathering information on the current market conditions and optimal pricing for AAPL, as well as reviewing the approved long-term investment, swing trading, and options trading strategies.

Action: R

I need to gather detailed information on the proposed trading strategies and execution plans for AAPL in order to evaluate the associated risks and suggest mitigation strategies.

Action: Delegate work to co-worker
Action Input: {"coworker": "Risk Advisor", "task": "Evaluate risks associated with the proposed trading strategies and execution plans for AAPL", "context": "Based on the information gathered, the long-term investment strategy for AAPL involves seeking firms with long-term, predictable profitability and low debt that trade at reasonable valuations. This aligns with Apple's strategy of creating products with customized hardware and software to differentiate their products. For swing trading, it may be beneficial to consider selling AAPL stocks at a profit within one year of purchase to capitalize on short-term capital gains. This strategy can be optimized by monitoring technical analysis indicators and investment potential. In terms of options trading, selling covered calls o

Final Answer: After analyzing the risks associated with the proposed trading strategies and execution plans for AAPL, it is important to consider factors such as investment style, market cap, technical indicators, fundamental analysis, risk tolerance, leverage, tax considerations, and more. Developing a well-considered trading strategy that aligns with the user's risk tolerance and market conditions is crucial for maximizing profitability and minimizing risk. It is essential to adhere to the trading plan diligently, periodically re-evaluate and tweak the strategy as needed, and avoid over-reliance on a single strategy to ensure consistent results.

> Finished chain.
 

After analyzing the risks associated with the proposed trading strategies and execution plans for AAPL, it is important to consider factors such as investment style, market cap, technical indicators, fundamental analysis, risk tolerance, leverage, tax considerations, and more. Developing a well-considered trading strateg

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for AAPL:

Introduction:
The purpose of this report is to evaluate the risks associated with the proposed trading strategies and execution plans for AAPL. The analysis will provide detailed insights into potential risks and suggest mitigation strategies to optimize profitability and minimize risk.

Risk Analysis:
1. Market Risk: The volatile nature of the stock market can pose a significant risk to the proposed trading strategies. Fluctuations in the market can impact the value of AAPL stocks and affect the overall profitability of the investments.

Mitigation Strategy: Implement stop-loss orders to limit potential losses in case of adverse market movements. Diversify the investment portfolio to reduce exposure to market risk.

2. Financial Risk: Changes in interest rates, inflation, and economic conditions can affect the financial performance of AAPL. High debt levels or cash flow issues can pose a financial risk to the investment.

Mitigation Strategy: Conduct thorough financial analysis of AAPL to assess its debt levels, cash flow position, and financial stability. Monitor economic indicators and financial reports to anticipate potential risks.

3. Operational Risk: Disruptions in supply chains, production issues, or regulatory changes can impact AAPL's operations and financial performance.

Mitigation Strategy: Stay informed about AAPL's operational activities and monitor news and updates related to the company. Implement risk management strategies to address operational risks proactively.

4. Liquidity Risk: Illiquidity in the market can hinder the ability to buy or sell AAPL stocks at desired prices, leading to potential losses.

Mitigation Strategy: Maintain sufficient liquidity in the investment portfolio to respond to market changes effectively. Consider alternative investment options to mitigate liquidity risk.

5. Legal and Compliance Risk: Changes in regulations, lawsuits, or legal issues can create uncertainties for AAPL and affect its stock price.

Mitigation Strategy: Stay abreast of regulatory changes and legal developments that may impact AAPL's operations. Ensure compliance with all legal requirements and conduct thorough due diligence before making investment decisions.

Conclusion:
In conclusion, a comprehensive risk analysis of the proposed trading strategies for AAPL reveals various risks that need to be managed effectively. By implementing the suggested mitigation strategies and staying informed about market conditions, investors can optimize profitability and minimize risk in their investment decisions. It is essential to continuously monitor and adjust the trading strategies to adapt to changing market dynamics and ensure long-term success.